In [ ]:
import xarray as xr
import numpy as np
from scipy.stats import gamma
import cftime

ds = xr.open_dataset('../Inputs/precip-detailed-noheight-ordered-sgreenland.KNMI-1950-2099.FGRN11.BN_RACMO2.3p2_CESM2_FGRN11.DD.nc')


yearly_groups = ds.groupby('time.year')

rlat_len = len(ds.rlat)
rlon_len = len(ds.rlon)

threshold = 0.76 / 86400 # change this based on what threshold you want to use

yearly_alphas = []
yearly_betas = []

for year, year_data in yearly_groups:
    
    precip_year = year_data.precip.values
    
    alpha_year = np.zeros((rlat_len, rlon_len))
    beta_year = np.zeros((rlat_len, rlon_len))
    
    for i in range(rlat_len):
        for j in range(rlon_len):
            data = precip_year[:, i, j]
            valid_data = data[~np.isnan(data) & (data > threshold)]
    
            if len(valid_data) > 10:
                    a, loc, scale = gamma.fit(valid_data, floc=0)
                    alpha_year[i, j] = a
                    beta_year[i, j] = scale
    
    yearly_alphas.append(alpha_year)
    yearly_betas.append(beta_year)

alpha = np.nanmean(np.array(yearly_alphas), axis=0)
beta = np.nanmean(np.array(yearly_betas), axis=0)

alpha = np.nan_to_num(alpha)
beta = np.nan_to_num(beta)

ds_out = xr.Dataset(
    {
        'alpha': (('rlat', 'rlon'), alpha),
        'beta': (('rlat', 'rlon'), beta)
    },
    coords={
        'rlat': ds['rlat'],
        'rlon': ds['rlon']
    }
)

# ds_out.to_netcdf('gamma_param_greenland_0.76mm.nc')
# print("Saved gamma parameters with yearly averaging, shape:", alpha.shape)

Saved gamma parameters with yearly averaging, shape: (121, 118)
